# Exploratory Analytics

##### Introduction

> Introduction to the dataset

###### AI-ML_Salaries
###### A Data Science portfolio project: Data Science job salaries from 2020 to 2022

###### About Dataset
The salaries are from [ai-jobs](https://salaries.ai-jobs.net/download/). Ai-jobs collects salary information anonymously from professionals all over the world in the AI/ML and Big Data space and makes it publicly available for anyone to use, share and play around with. The data is being updated regularly with new data coming in, usually on a weekly basis.
The primary goal is to have data that can provide better guidance in regards to what's being paid globally. So newbies, experienced pros, hiring managers, recruiters and also startup founders or people wanting to make a career switch can make better informed decisions.

The dataset contains one table structured as follow:
- **work_year**: The year the salary was paid.
- **experience_level**: The experience level in the job during the year with the following possible values:
    - **EN**: Entry-level / Junior
    - **MI**: Mid-level / Intermediate
    - **SE**: Senior-level / Expert
    - **EX**: Executive-level / Director
- **employment_type**: The type of employement for the role:
    - **PT**: Part-time
    - **FT**: Full-time
    - **CT**: Contract
    - **FL**: Freelance
- **job_title**: The role worked in during the year.
- **salary**: The total gross salary amount paid.
- **salary_currency**: The currency of the salary paid as an ISO 4217 currency code.
- **salaryinusd**: The salary in USD (FX rate divided by avg. USD rate for the respective year via fxdata.foorilla.com).
- **employee_residence**: Employee's primary country of residence in during the work year as an ISO 3166 country code.
- **remote_ratio**: The overall amount of work done remotely, possible values are as follows:
    - **0**: No remote work (less than 20%)
    - **50**: Partially remote
    - **100**: Fully remote (more than 80%)
- **company_location**: The country of the employer's main office or contracting branch as an ISO 3166 country code.
- **company_size**: The average number of people that worked for the company during the year:
    - **S**: less than 50 employees (small)
    - **M**: 50 to 250 employees (medium)
    - **L**: more than 250 employees (large)

##### Preliminary Wrangling

In [42]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
%matplotlib inline

> - Loading in dataset with pandas
> - Describing the dataset
> - Printing out some parts of the table

In [43]:
# importing or loading in the dataset using pandas, here I am using a copy of salaries.csv (salaries_usable.csv)
salaries_df = pd.read_csv('salaries_usable.csv')

# shape of the dataset
Shape = salaries_df.shape
columns = salaries_df.shape[1]
rows = salaries_df.shape[0]

print("This dataset contains {} rows and {} columns, hence {}".format(rows, columns, Shape))

# print out some part of dataset

# Head
print(salaries_df.head(5))

# 10 random samples
print(salaries_df.sample(n = 10))

# Tail
print(salaries_df.tail())

This dataset contains 1332 rows and 11 columns, hence (1332, 11)
   work_year experience_level employment_type                  job_title  \
0       2022               MI              FT  Machine Learning Engineer   
1       2022               MI              FT  Machine Learning Engineer   
2       2022               MI              FT             Data Scientist   
3       2022               MI              FT             Data Scientist   
4       2022               MI              FT             Data Scientist   

   salary salary_currency  salary_in_usd employee_residence  remote_ratio  \
0  130000             USD         130000                 US             0   
1   90000             USD          90000                 US             0   
2  120000             USD         120000                 US           100   
3  100000             USD         100000                 US           100   
4   85000             USD          85000                 US           100   

  company_locat

##### Assesssment and Cleaning of the dataset

###### **Assessment**

In [44]:
# Info
print(salaries_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           1332 non-null   int64 
 1   experience_level    1332 non-null   object
 2   employment_type     1332 non-null   object
 3   job_title           1332 non-null   object
 4   salary              1332 non-null   int64 
 5   salary_currency     1332 non-null   object
 6   salary_in_usd       1332 non-null   int64 
 7   employee_residence  1332 non-null   object
 8   remote_ratio        1332 non-null   int64 
 9   company_location    1332 non-null   object
 10  company_size        1332 non-null   object
dtypes: int64(4), object(7)
memory usage: 114.6+ KB
None


In [45]:
# Display summary statistics for numeric columns
print(salaries_df.describe())

         work_year        salary  salary_in_usd  remote_ratio
count  1332.000000  1.332000e+03    1332.000000   1332.000000
mean   2021.716216  2.377124e+05  123374.658408     63.851351
std       0.562228  1.077369e+06   65945.872172     45.263587
min    2020.000000  2.324000e+03    2324.000000      0.000000
25%    2022.000000  8.000000e+04   75593.000000      0.000000
50%    2022.000000  1.300000e+05  120000.000000    100.000000
75%    2022.000000  1.751000e+05  164997.000000    100.000000
max    2022.000000  3.040000e+07  600000.000000    100.000000


In [46]:
# Duplicated values
# True: duplicated
# False: not duplicated, unique
print(salaries_df.duplicated().sum())
print((~salaries_df.duplicated()).sum())
print(salaries_df.shape[0] - salaries_df.duplicated().sum())
# salaries_df.duplicated().value_counts()
            # output 
            # False 1069
            # True 263

263
1069
1069


In [47]:
# Display entries for each year for the dataset and duplicates

# get the duplicate data
duplicates = salaries_df[salaries_df.duplicated()]

# print the year entries for the whole dataset
print(salaries_df.work_year.value_counts())

# print the year entries for duplicates
print(duplicates.work_year.value_counts())


2022    1029
2021     228
2020      75
Name: work_year, dtype: int64
2022    260
2021      2
2020      1
Name: work_year, dtype: int64


###### **Cleaning**

In [48]:
# copy the dataframe for safety
salaries_clean = salaries_df.copy()

In [49]:
# Function to convert datatype to categorical (ordinal and nominal)
def to_category(var: str, ordered: bool, df: pd.DataFrame, categories: list):
    """ Converts type to category
            parameters:
            var: string, the name of the column to convert type
            ordered: boolean, If false, then the categorical is treated as unordered
            df: dataframe, 
            categories: the ordered list of the category
            
            return: the converted series 
    
    """
    # Note: based on the pandas version, you need to code differently
    pd_ver = pd.__version__.split(".")
    if (int(pd_ver[0]) > 0) or (int(pd_ver[1]) >= 21): # v0.21 or later
        categorize = pd.api.types.CategoricalDtype(ordered = ordered, categories = categories)
        df[var] = df[var].astype(categorize)
    else: # pre-v0.21
        df[var] = df[var].astype('category', ordered = ordered,categories = categories)
    return df[var]

In [50]:
# Create category type
experience = ['EN', 'MI', 'SE', 'EX']
company = ['S', 'M', 'L']
employment_type = ['PT', 'FT', 'CT', 'FL']
remote = [0, 50, 100]
year = [2020, 2021, 2022]

salaries_clean['experience_level'] = to_category('experience_level', True, salaries_clean, experience) # ordered category
salaries_clean['company_size'] = to_category('company_size', True, salaries_clean, company) # ordered category
salaries_clean['employment_type'] = to_category('employment_type', False, salaries_clean, employment_type) # unordered category
salaries_clean['remote_ratio'] = to_category('remote_ratio', True, salaries_clean, remote) # ordered category
salaries_clean['work_year'] = to_category('work_year', True, salaries_clean, year) # ordered category

# Test types were converted. Dtype should be category
salaries_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           1332 non-null   category
 1   experience_level    1332 non-null   category
 2   employment_type     1332 non-null   category
 3   job_title           1332 non-null   object  
 4   salary              1332 non-null   int64   
 5   salary_currency     1332 non-null   object  
 6   salary_in_usd       1332 non-null   int64   
 7   employee_residence  1332 non-null   object  
 8   remote_ratio        1332 non-null   category
 9   company_location    1332 non-null   object  
 10  company_size        1332 non-null   category
dtypes: category(5), int64(2), object(4)
memory usage: 69.9+ KB


In [51]:
# Duplicates still exist?
print(salaries_clean.duplicated().sum())

263


In [52]:
# Delete duplicates and reset index
salaries_clean.drop_duplicates(inplace= True, ignore_index=True)

# Test duplicates were deleted. result should be 0
print(salaries_clean.duplicated().sum())

# The new dimention of the dataset is = non duplicates
print(salaries_clean.shape)

0
(1069, 11)


In [ ]:
# Now let's save the non-duplicates into another file as a cleaned data
salaries_clean.to_csv('cleaned_salaries_usable.csv', index=False)